<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install yfinance==0.2.59

  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.31.1
    Uninstalling protobuf-6.31.1:
      Successfully uninstalled protobuf-6.31.1


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.65
    Uninstalling yfinance-0.2.65:
      Successfully uninstalled yfinance-0.2.65
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-07-09 09:18:38
-------------------
qualified stocks: 96
with latest results: 31
still star stocks: 20
-------------------
Initial Investment:  1.17 C
CY Investment:  1.44 C
Reserve:  9.00 K
Current:  1.37 C
-------------------
Today PnL: 3.01 K (0.02%)
Current PnL: -13.22 L (-9.21%)
CY Booked + Current PnL: -6.26 L (-4.36%)
-------------------
Total profit:  4.73 L
Total loss:  -17.95 L
-------------------
Total Booked + Current PnL: 20.72 L (17.78%)
Total Booked PnL: 33.94 L (29.12%)
Curr Year Booked PnL: 6.96 L (5.08%)
Prev Year Booked PnL: 26.98 L (23.15%)
Est FTT:  2.09 C
Est FTT PnL: 71.69 L (52.37%)
Deployed:  1.17 C
Current:  1.37 C
CAGR/XIRR %: 10.84%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'AvgCost', 'FTT', 'Dev%_PE', 'Conviction', 'RSP', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
4,APOLLOHOSP,6904.43,8285.00,-16.74,H-LC,90.62,113362.0,9796.0,10917.0,-0.10,9.46,9.63,20.00,32.0,0.90,0.83,24.86,X40N,BTT,HEALTHCARE
39,HINDALCO,651.95,761.55,-20.73,H-LC,84.38,109832.0,5520.0,12016.0,0.12,5.29,10.94,16.81,27.0,0.46,0.80,22.19,X5K,ATH,METALS
92,VIPIND,488.80,489.00,-213.57,H-SC,98.96,81880.0,-13436.0,13477.0,-0.33,-14.10,16.46,0.04,157.0,-1.00,0.60,61.50,OX40N,NTT,MISC
71,SBIN,760.30,863.00,-11.51,M-LC,30.21,214809.0,13330.0,13877.0,-0.25,6.62,6.46,13.51,77.0,0.96,1.57,20.05,XY25,NTT,BANKS
87,UJJIVANSFB,52.77,53.00,47.02,M-SC,19.27,127251.0,-15228.0,15843.0,0.28,-10.69,12.45,0.44,247.0,-0.96,0.93,50.19,OX40N,NTT,BANKS


In [10]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
12,BANKINDIA,116.91,190.00,-31.29,M-MC,9.38,179039.0,-769.0,113188.0,-0.47,-0.43,63.22,62.52,173.0,-0.01,1.31,33.51,XR,NTT,BANKS
38,HEROMOTOCO,4311.81,6039.03,-6.19,H-MC,31.25,151116.0,203.0,60250.0,0.40,0.13,39.87,40.06,103.0,0.00,1.10,23.60,AR,ATH,AUTO
60,NATIONALUM,189.63,247.44,-50.15,H-MC,51.04,100117.0,-387.0,31026.0,-0.53,-0.38,30.99,30.49,97.0,-0.01,0.73,31.89,MH,ATH,METALS


In [11]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
52,JPPOWER,18.73,26.20,-13.49,L-SC,100.00,173800.0,31958.0,24610.0,2.50,22.53,14.16,39.88,262.0,1.30,1.27,76.13,XY24,NTT,POWER
55,LALPATHLAB,2804.19,3545.00,-29.67,H-SC,66.67,161854.0,7624.0,33115.0,2.18,4.94,20.46,26.42,127.0,0.23,1.18,27.64,X40N,NTT,HEALTHCARE
91,VBL,492.64,672.28,-17.67,H-LC,26.04,249901.0,-15139.0,111781.0,1.39,-5.71,44.73,36.46,25.0,-0.14,1.83,6.73,X40N,ATH,FMCG
16,BLUSPRING,226.45,85.85,NaN,NaN,58.33,18801.0,-30792.0,0.0,1.33,-62.09,0.00,-62.09,270.0,-inf,0.14,26.08,XY24,ATH,MISC
69,SAMMAANCAP,170.35,326.00,-165.91,M-SC,39.58,76020.0,-26190.0,119579.0,1.30,-25.62,157.30,91.37,212.0,-0.22,0.56,23.41,XY25,NTT,FINANCE


In [12]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
0,5PAISA,564.75,565.0,140.38,H-SC,7.29,129653.0,-57279.0,57358.0,-5.90,-30.64,44.24,0.04,149.0,-1.00,0.95,24.03,OX40N,NTT,FINANCE
88,UNIONBANK,123.87,163.0,-6.01,M-LC,5.21,162591.0,21751.0,22746.0,-4.83,15.44,13.99,31.59,88.0,0.96,1.19,41.23,XY24,NTT,BANKS
65,RELAXO,902.64,1176.0,-30.47,H-SC,92.71,62899.0,-52639.0,87631.0,-0.97,-45.56,139.32,30.28,131.0,-0.60,0.46,22.88,X40N,NTT,FOOTWEAR
6,ASIANTILES,75.41,137.0,7066.67,M-SC,69.79,78633.0,-15177.0,91796.0,-0.93,-16.18,116.74,81.67,245.0,-0.17,0.57,52.28,XR,NTT,CERAMICS
33,GREENPANEL,375.16,537.0,213.31,M-SC,82.29,139934.0,-43144.0,122120.0,-0.83,-23.57,87.27,43.14,217.0,-0.35,1.02,31.63,XY24,NTT,MISC


In [13]:
# OX40N stocks
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
87,UJJIVANSFB,52.77,53.0,47.02,M-SC,19.27,127251.0,-15228.0,15843.0,0.28,-10.69,12.45,0.44,247.0,-0.96,0.93,50.19,OX40N,NTT,BANKS
79,SYMPHONY,1306.42,1306.0,-28.80,M-SC,28.12,151790.0,-19351.0,19293.0,0.41,-11.31,12.71,-0.03,191.0,-1.00,1.11,9.10,OX40N,NTT,DURABLES
92,VIPIND,488.80,489.0,-213.57,H-SC,98.96,81880.0,-13436.0,13477.0,-0.33,-14.10,16.46,0.04,157.0,-1.00,0.60,61.50,OX40N,NTT,MISC
46,INDIGOPNTS,1407.73,1408.0,128.24,H-SC,93.75,149569.0,-24990.0,25023.0,0.49,-14.32,16.73,0.02,133.0,-1.00,1.09,29.70,OX40N,NTT,PAINTS
81,TATAELXSI,7332.28,7332.0,-17.82,X-MC,10.42,86205.0,-16447.0,16439.0,-0.23,-16.02,19.07,-0.00,59.0,-1.00,0.63,31.04,OX40N,NTT,IT


In [14]:
# top 5 to accumulate
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
83,TCS,3794.03,4998.00,-18.38,X-LC,37.50,265613.0,-30321.0,124227.0,-0.03,-10.25,46.77,31.73,1.0,-0.24,1.94,6.31,X40,BTT,IT
48,INFY,1461.46,2275.00,-9.19,X-LC,73.96,291734.0,30133.0,115497.0,-0.54,11.52,39.59,55.67,2.0,0.26,2.13,18.33,X40,BTT,IT
40,HINDUNILVR,2413.81,2723.00,-13.47,X-LC,63.54,271130.0,783.0,33837.0,1.17,0.29,12.48,12.81,5.0,0.02,1.98,13.39,XY25,NTT,FMCG
17,BRITANNIA,4983.33,6446.05,16.86,X-LC,62.50,267858.0,38625.0,28661.0,-0.27,16.85,10.70,29.35,10.0,1.35,1.96,27.27,XY25,ATH,FMCG
74,SIEMENS,4406.90,7969.85,-3.07,X-LC,23.96,81960.0,-28212.0,117285.0,-0.56,-25.61,143.10,80.85,11.0,-0.24,0.60,20.71,AR,ATH,ELECTRICAL


In [15]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
32,ENRIN,1377.95,3064.50,NaN,NaN,94.79,76612.0,42163.0,0.0,0.58,122.39,0.00,122.40,274.0,inf,0.56,17.15,AR,ATH,ELECTRICAL
28,DIGITIDE,188.38,237.72,NaN,NaN,47.92,51349.0,10094.0,714.0,0.66,24.47,1.39,26.19,272.0,14.14,0.38,20.89,XY24,ATH,IT
52,JPPOWER,18.73,26.20,-13.49,L-SC,100.00,173800.0,31958.0,24610.0,2.50,22.53,14.16,39.88,262.0,1.30,1.27,76.13,XY24,NTT,POWER


In [16]:
# top 5 to monitor
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
68,SAIL,130.64,228.00,93.57,M-MC,44.79,232556.0,7594.0,160068.0,1.18,3.38,68.83,74.53,189.0,0.05,1.70,35.02,XY24,BTT,STEEL
54,KPIGREEN,497.21,731.82,25.90,H-SC,33.85,128646.0,3349.0,55768.0,0.65,2.67,43.35,47.19,143.0,0.06,0.94,60.94,MH,ATH,POWER
20,CAMS,3643.00,5250.99,2.76,H-SC,19.27,115360.0,13356.0,31666.0,-0.04,13.09,27.45,44.14,126.0,0.42,0.84,31.46,X40N,ATH,MISC
95,WIPRO,243.46,420.00,-2.41,M-LC,88.54,167183.0,16238.0,93221.0,0.71,10.76,55.76,72.51,71.0,0.17,1.22,15.16,XR,NTT,IT
15,BLUESTARCO,1646.70,2337.55,-3.59,H-SC,96.88,180940.0,16270.0,52816.0,-0.04,9.88,29.19,41.95,124.0,0.31,1.32,18.08,X40N,ATH,AC


In [17]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
32,ENRIN,1377.95,3064.50,NaN,NaN,94.79,76612.0,42163.0,0.0,0.58,122.39,0.00,122.40,274.0,inf,0.56,17.15,AR,ATH,ELECTRICAL
94,WHIRLPOOL,1167.49,2270.00,-40.58,M-SC,29.17,195745.0,24124.0,137942.0,-0.11,14.06,70.47,94.43,201.0,0.17,1.43,44.47,XR,NTT,DURABLES
95,WIPRO,243.46,420.00,-2.41,M-LC,88.54,167183.0,16238.0,93221.0,0.71,10.76,55.76,72.51,71.0,0.17,1.22,15.16,XR,NTT,IT
45,INDIAMART,2327.09,4850.92,-47.70,H-SC,77.08,136162.0,12826.0,120939.0,0.61,10.40,88.82,108.45,123.0,0.11,1.00,35.08,AR,ATH,MISC
54,KPIGREEN,497.21,731.82,25.90,H-SC,33.85,128646.0,3349.0,55768.0,0.65,2.67,43.35,47.19,143.0,0.06,0.94,60.94,MH,ATH,POWER


In [18]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
16,BLUSPRING,226.45,85.85,NaN,NaN,58.33,18801.0,-30792.0,0.0,1.33,-62.09,0.00,-62.09,270.0,-inf,0.14,26.08,XY24,ATH,MISC
28,DIGITIDE,188.38,237.72,NaN,NaN,47.92,51349.0,10094.0,714.0,0.66,24.47,1.39,26.19,272.0,14.14,0.38,20.89,XY24,ATH,IT
24,COFFEEDAY,59.14,80.00,-55.73,L-SC,64.58,65011.0,-48538.0,88590.0,0.92,-42.75,136.27,35.27,265.0,-0.55,0.48,59.12,XR,NTT,HOTELS
32,ENRIN,1377.95,3064.50,NaN,NaN,94.79,76612.0,42163.0,0.0,0.58,122.39,0.00,122.40,274.0,inf,0.56,17.15,AR,ATH,ELECTRICAL
58,MASFIN,326.60,399.50,-18.25,H-SC,65.62,92805.0,-5175.0,27043.0,-0.72,-5.28,29.14,22.32,145.0,-0.19,0.68,34.60,XR,ATH,FINANCE


In [19]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
52,JPPOWER,18.73,26.20,-13.49,L-SC,100.00,173800.0,31958.0,24610.0,2.50,22.53,14.16,39.88,262.0,1.30,1.27,76.13,XY24,NTT,POWER
92,VIPIND,488.80,489.00,-213.57,H-SC,98.96,81880.0,-13436.0,13477.0,-0.33,-14.10,16.46,0.04,157.0,-1.00,0.60,61.50,OX40N,NTT,MISC
3,ALKYLAMINE,2347.98,4546.37,7.94,X-SC,97.92,100517.0,-446.0,94979.0,-0.78,-0.44,94.49,93.63,67.0,-0.00,0.73,49.72,SR,ATH,CHEMICALS
15,BLUESTARCO,1646.70,2337.55,-3.59,H-SC,96.88,180940.0,16270.0,52816.0,-0.04,9.88,29.19,41.95,124.0,0.31,1.32,18.08,X40N,ATH,AC
5,ASIANPAINT,2961.56,4250.00,-14.02,H-LC,95.83,170748.0,-30638.0,118260.0,1.06,-15.21,69.26,43.51,31.0,-0.26,1.25,18.86,X40,BTT,PAINTS


In [20]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,18.95
1,25,41.94
2,50,70.57


In [21]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    43.39
LC    33.93
MC    21.61
Name: CurrAlloc%, dtype: float64

In [22]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     30.95
X40      12.97
XY25     12.08
X40N     11.95
XR       10.01
AR        8.18
OX40N     8.14
MH        1.67
X5K       1.44
X200      1.41
SR        1.21
Name: CurrAlloc%, dtype: float64

In [23]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    20.45
M-SC    15.83
H-LC    14.16
X-LC    12.91
H-MC     9.57
X-MC     7.61
M-LC     5.82
X-SC     4.44
M-MC     4.07
L-SC     2.67
L-LC     1.04
L-MC     0.36
Name: CurrAlloc%, dtype: float64

In [24]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,15.28,-0.21,33.70
IT,12.12,-9.60,63.47
FINANCE,8.47,-12.52,53.35
BANKS,7.64,-9.23,53.80
MISC,5.85,-20.25,76.25
PAINTS,5.72,-11.47,33.44
ELECTRICAL,5.46,1.46,44.86
HEALTHCARE,4.66,-1.01,30.51
AUTO,4.31,-14.67,65.09


In [25]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,2657041.0,26
XR,1089970.0,13
AR,1026914.0,10
X40,681490.0,10
X40N,526242.0,11
XY25,467256.0,8
OX40N,317233.0,11
SR,177363.0,2
MH,86794.0,2


In [26]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,1985107.0,20
M-SC,1776962.0,21
H-LC,604531.0,12
X-LC,542746.0,8
H-MC,524365.0,8
M-MC,413577.0,3
X-MC,411952.0,6
X-SC,309550.0,5
M-LC,265060.0,4


In [27]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                 
H-SC       XY24      889768.0      6
M-SC       XY24      811372.0      7
H-SC       AR        523555.0      3
M-SC       XR        341136.0      4
M-MC       XY24      300389.0      2
H-MC       XY24      295568.0      3
X-LC       X40       275881.0      3
H-SC       XR        214930.0      3
           X40N      205228.0      4
H-LC       X40N      193356.0      4
M-SC       AR        186000.0      2
           OX40N     165652.0      5
M-LC       XY24      157962.0      2
X-MC       X40       153890.0      2
H-LC       X40       152184.0      3
L-SC       XR        150431.0      2
X-SC       XR        123071.0      1
M-SC       XY25      119579.0      1
X-LC       AR        117285.0      1
M-MC       XR        113188.0      1
X-MC       XY25      102663.0      1
X-LC       XY25      101437.0      3
H-SC       OX40N      95858.0      3
X-SC       SR         94979.0      1
M-LC       XR         93221.0      1
X-MC       XY24       86154.0      1
M-SC       SR         82384.0      1
H-MC       XY25       82116.0      1
H-LC       AR         81839.0      1
           X5K        70869.0      2
M-SC       X40        70839.0      1
H-LC       X200       67995.0      1
H-MC       AR         60250.0      1
L-SC       AR         57985.0      1
H-SC       MH         55768.0      1
L-MC       XR         53993.0      1
X-MC       X40N       52806.0      1
X-SC       XY24       52216.0      1
X-LC       X40N       48143.0      1
L-LC       XY25       47584.0      1
X-SC       OX40N      39284.0      2
H-LC       XY24       38288.0      1
H-MC       MH         31026.0      1
           X40        28696.0      1
           X40N       26709.0      1
L-SC       XY24       24610.0      1
X-MC       OX40N      16439.0      1
M-LC       XY25       13877.0      1

In [28]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 28.0 seconds
